In [2]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import xarray as xr
import pandas as pd
import os
import sys
import seaborn as sns
import random 
import scipy as sc
import statsmodels.api as sm
import scipy.stats as stats
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy as cp
import stormeunice as eun
import multiprocessing
import glob
import dask

sys.path.append('/home/e/ermis/moarpalettes')
from moarpalettes import get_palette

sns.set_theme(style="white")
sns.set_style("white")
%matplotlib inline

random.seed(10)

In [3]:
# Import track data
P1 = multiprocessing.Pool(40)
ifs_tracks = P1.map(eun.lagrangian.Lagrange.import_medr_tracks_TE, 
                    glob.glob('/home/l/leach/Attribution/NA-Storms/Nick/Scripts/tracking/TE/TEStitch_[!2]*'))
P1.close()

ifs_tracks = pd.concat(ifs_tracks)
ifs_tracks['date'] = pd.to_datetime(ifs_tracks.loc[:,['year','month','day','hour']])

# add era track in
era_track = pd.read_csv('/home/l/leach/Attribution/NA-Storms/Nick/Scripts/tracking/TE/TEStitch_2022_0',
                        skipinitialspace=True)

era_track['expid'] = 'era5'
era_track['experiment'] = 'era5'
era_track['inidate'] = pd.to_datetime('2022-01-01')
era_track['number'] = 0
era_track['date'] = pd.to_datetime(era_track.loc[:,['year','month','day','hour']])

all_tracks = pd.concat([ifs_tracks,era_track])

eunice_track = era_track.query('track_id==5')


In [4]:
ifs_tracks_filter = ifs_tracks.groupby(['track_id','inidate','experiment','number']).agg(dict(date=min, day=np.size, lon=lambda x: x.iloc[0], lat=lambda x: x.iloc[0], msl=min))
eunice_filter = eunice_track.groupby(['track_id','inidate','experiment','number']).agg(dict(date=min, day=np.size, lon=lambda x: x.iloc[0], lat=lambda x: x.iloc[0], msl=min)).iloc[0]

# starttime criterion - must be detected within 12 hours (before or after) the ERA5 track detectino time
starttime_crit = (ifs_tracks_filter.date - eunice_filter.date).dt.total_seconds().abs()<(12*60*60)
# starting location criterion - must start within 10 degrees of the ERA5 track starting point
startloc_crit = np.sqrt((ifs_tracks_filter.lon-eunice_filter.lon)**2+(ifs_tracks_filter.lat-eunice_filter.lat)**2)<10
# depth criterion - must reach a depth of at least 980 hPa
minmsl_crit = ifs_tracks_filter.msl < 98000

ifs_tracks_filtered = ifs_tracks_filter.loc[starttime_crit&startloc_crit&minmsl_crit]
ifs_eunice_list = ifs_tracks.set_index(['track_id','inidate','experiment','number']).sort_index().loc[ifs_tracks_filtered.index].reset_index()

# For members with more than one Eunice-like track: keep the ones with the longest track? 
multi_track_members = ifs_eunice_list.groupby(['inidate','experiment','number','track_id']).agg(dict(year = lambda x: 1)).groupby(['inidate','experiment','number']).count().query('year > 1')

for idx in multi_track_members.index:
    track_dists = ifs_eunice_list.query('inidate=="{}" & experiment=="{}" & number=={}'.format(*idx)).groupby('track_id').apply(lambda x: eun.lagrangian.Lagrange.eunice_dist(x,eunice_track=eunice_track))
    num_reject = track_dists.idxmax()
    ifs_eunice_list_idx = ifs_eunice_list.query('inidate=="{}" & experiment=="{}" & number=={} & track_id=={}'.format(*idx,num_reject)).index
    ifs_eunice_list = ifs_eunice_list.drop(ifs_eunice_list_idx)


In [5]:
ERA5_tracks = pd.read_hdf('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/postproc/tracks/ERA5/EU025/sfc/2022.h5')
ERA5_eunice_track = ERA5_tracks.query('uid=="202202079"')

ERA5_data = xr.open_dataset('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/ERA5/EU025/sfc/2022.nc').sel(time=ERA5_eunice_track.date.values)
ERA5_data['centroid_lon'] = ('time',(ERA5_eunice_track.long*4).round()/4)
ERA5_data['centroid_lat'] = ('time',(ERA5_eunice_track.lat*4).round()/4)

ERA5_eunice_stormframe = ERA5_data.groupby('time').apply(eun.lagrangian.Lagrange.lagrangian_frame)

In [8]:
ERA5_tracks.groupby(['uid']).describe()

Dp                                                            \
          count       mean         std         min         25%        50%   
uid                                                                         
202202000  20.0  40.196518  162.834281 -281.132062  -33.862215  37.831422   
202202001   3.0  74.788878   78.720013    0.000000   33.720873  67.441746   
202202002  11.0  47.737023   45.134846    0.000000   17.597914  33.422430   
202202003  10.0  74.071653  266.456212 -242.073234 -151.929868  71.090346   
202202004   3.0 -60.713774   49.777809  -98.078542  -88.967500 -79.856457   
...         ...        ...         ...         ...         ...        ...   
202202116   2.0 -65.175494   92.172068 -130.350989  -97.763242 -65.175494   
202202117   1.0   0.000000         NaN    0.000000    0.000000   0.000000   
202202118   1.0   0.000000         NaN    0.000000    0.000000   0.000000   
202202119   1.0   0.000000         NaN    0.000000    0.000000   0.000000   
202202120   2.0  22.659772   32.045756    0.000000   11.329886  22.659772   

                                   DpDt              ...  mon      dDist  \
                  75%         max count        mean  ...  75%  max count   
uid                                                  ...                   
202202000  102.292753  556.067660  19.0  506.554027  ...  2.0  2.0  20.0   
202202001  112.183318  156.924890   2.0  888.382189  ...  2.0  2.0   3.0   
202202002   75.126178  138.887684  10.0  422.754103  ...  2.0  2.0  11.0   
202202003  236.990752  575.256394   9.0  683.748202  ...  2.0  2.0  10.0   
202202004  -42.031389   -4.206321   2.0 -308.864573  ...  2.0  2.0   3.0   
...               ...         ...   ...         ...  ...  ...  ...   ...   
202202116  -32.587747    0.000000   1.0 -942.024505  ...  2.0  2.0   2.0   
202202117    0.000000    0.000000   0.0         NaN  ...  2.0  2.0   1.0   
202202118    0.000000    0.000000   0.0         NaN  ...  2.0  2.0   1.0   
202202119    0.000000    0.000000   0.0         NaN  ...  2.0  2.0   1.0   
202202120   33.989657   45.319543   1.0  576.417175  ...  2.0  2.0   2.0   

                                                                             \
                 mean         std   min         25%         50%         75%   
uid                                                                           
202202000   90.494864   94.057260   0.0   35.355339   55.901699   98.854086   
202202001   57.929440   52.714040   0.0   35.355339   70.710678   86.894159   
202202002   29.823676   27.777746   0.0    0.000000   25.000000   55.901699   
202202003  146.553453   75.623964   0.0  115.721421  145.773797  185.970652   
202202004  159.106372  194.043149  25.0   47.855339   70.710678  226.159558   
...               ...         ...   ...         ...         ...         ...   
202202116   97.628121  138.067013   0.0   48.814060   97.628121  146.442181   
202202117    0.000000         NaN   0.0    0.000000    0.000000    0.000000   
202202118    0.000000         NaN   0.0    0.000000    0.000000    0.000000   
202202119    0.000000         NaN   0.0    0.000000    0.000000    0.000000   
202202120    0.000000    0.000000   0.0    0.000000    0.000000    0.000000   

                       
                  max  
uid                    
202202000  335.410197  
202202001  103.077641  
202202002   75.000000  
202202003  285.043856  
202202004  381.608438  
...               ...  
202202116  195.256242  
202202117    0.000000  
202202118    0.000000  
202202119    0.000000  
202202120    0.000000  

[121 rows x 256 columns]

In [ ]:
%%capture 
# Ignore outputs

# Import field data, Lagrangian fram
curr_storms = xr.open_mfdataset('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/pf/*.nc',
                                preprocess=lambda x: eun.lagrangian.Lagrange.preproc_to_stormframe(x, ifs_eunice_list=ifs_eunice_list),
                                combine='nested',
                                concat_dim=['inidate'],
                                parallel=True, 
                                engine='netcdf4')
pi_storms = xr.open_mfdataset('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/pi/EU025/sfc/pf/*.nc',
                              preprocess=lambda x: eun.lagrangian.Lagrange.preproc_to_stormframe(x, ifs_eunice_list=ifs_eunice_list),
                              combine='nested',
                              concat_dim=['inidate'],
                              parallel=True, 
                              engine='netcdf4')
incr_storms = xr.open_mfdataset('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/incr/EU025/sfc/pf/*.nc',
                                preprocess=lambda x: eun.lagrangian.Lagrange.preproc_to_stormframe(x, ifs_eunice_list=ifs_eunice_list),
                                combine='nested',
                                concat_dim=['inidate'],
                                parallel=True, 
                                engine='netcdf4')